In [1]:
import pandas as pd
import scipy.stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv(r'D:\data_2.csv')

df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка дома
1,1,-4024.803754,36,среднее образование,1,женат / замужем,0,F,сотрудник,0,112080.014102,покупка автомобиля
2,0,-5623.422610,33,Среднее образование,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка дома
3,3,-4124.747207,32,среднее образование,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее образование,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [4]:
df.duplicated().sum()

54

In [5]:
df = df.drop_duplicates().reset_index(drop=True)

In [6]:
df.duplicated().sum()

0

In [7]:
df['education'] = df['education'].str.lower()
df['education'].value_counts()

education
среднее образование      15188
высшее                    5251
неоконченное высшее        744
начальное образование      282
ученая степень               6
Name: count, dtype: int64

In [8]:
df['family_status'].value_counts()

family_status
женат / замужем          12344
гражданский брак          4163
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: count, dtype: int64

In [9]:
df['family_status'] = df['family_status'].str.lower()
df['family_status'].value_counts()

family_status
женат / замужем          12344
гражданский брак          4163
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: count, dtype: int64

In [10]:
df['income_type'].value_counts()

income_type
сотрудник          11091
компаньон           5080
пенсионер           3837
госслужащий         1457
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: count, dtype: int64

In [11]:
df['gender'].value_counts()

gender
F      14189
M       7281
XNA        1
Name: count, dtype: int64

In [12]:
df['purpose'] = df['purpose'].str.lower()
df['purpose'].value_counts()

purpose
строительство недвижимости                 1244
покупка подержанного автомобиля             964
свадебная церемония                         793
проведение свадьбы                          773
сыграть свадьбу                             769
сделки с недвижимостью                      675
купить коммерческую недвижимость            662
покупка недвижимости для сдачи в аренду     652
сделки с жильем                             652
сделки с жилой недвижимостью                650
покупка дома                                646
жилье                                       646
покупка дома для своей семьи                638
строительство собственной недвижимости      635
недвижимость                                633
сделки со своей недвижимостью               627
купить недвижимость                         621
покупка собственного дома                   620
ремонт недвижимости                         607
покупка жилой недвижимости                  606
покупка собственного автомобиля 

In [13]:
def categ(purpose):
    if 'свадь' in purpose:
        return 'свадьба'
    elif 'маш' in purpose or 'авто' in purpose:
        return 'автомобиль'
    elif 'образ' in purpose:
        return 'образование'
    return 'недвижимость'

In [14]:
df['purpose'] = df['purpose'].apply(categ)

In [15]:
df['purpose'].value_counts()

purpose
недвижимость    11607
автомобиль       4308
образование      4014
свадьба          1542
Name: count, dtype: int64

In [16]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,недвижимость
1,1,-4024.803754,36,среднее образование,1,женат / замужем,0,F,сотрудник,0,112080.014102,автомобиль
2,0,-5623.422610,33,среднее образование,1,женат / замужем,0,M,сотрудник,0,145885.952297,недвижимость
3,3,-4124.747207,32,среднее образование,1,женат / замужем,0,M,сотрудник,0,267628.550329,образование
4,0,340266.072047,53,среднее образование,1,гражданский брак,1,F,пенсионер,0,158616.077870,свадьба


In [17]:
df_education = df[['education_id', 'education']].drop_duplicates().reset_index(drop=True)

df_education

,education_id,education
0,0,высшее
1,1,среднее образование
2,2,неоконченное высшее
3,3,начальное образование
4,4,ученая степень


In [18]:
df_family = df[['family_status_id', 'family_status']].drop_duplicates().reset_index(drop=True)

df_family

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


In [19]:
df = df.drop(['family_status', 'education'], axis = 1)

In [20]:
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,0,0,F,сотрудник,0,253875.639453,недвижимость
1,1,-4024.803754,36,1,0,F,сотрудник,0,112080.014102,автомобиль
2,0,-5623.422610,33,1,0,M,сотрудник,0,145885.952297,недвижимость
3,3,-4124.747207,32,1,0,M,сотрудник,0,267628.550329,образование
4,0,340266.072047,53,1,1,F,пенсионер,0,158616.077870,свадьба


In [21]:
df = df.loc[df['gender'] != 'XNA']

df['gender'].value_counts()

gender
F    14189
M     7281
Name: count, dtype: int64

In [22]:
df['days_employed'] = df['days_employed'].abs()

df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,F,сотрудник,0,253875.639453,недвижимость
1,1,4024.803754,36,1,0,F,сотрудник,0,112080.014102,автомобиль
2,0,5623.422610,33,1,0,M,сотрудник,0,145885.952297,недвижимость
3,3,4124.747207,32,1,0,M,сотрудник,0,267628.550329,образование
4,0,340266.072047,53,1,1,F,пенсионер,0,158616.077870,свадьба


In [23]:
min(df['days_employed'])

24.14163324048118

In [24]:
max(df['days_employed'])

401755.40047533

In [25]:
df[df['days_employed'] >= 20000]['days_employed'].count()

3445

In [26]:
df[df['days_employed'] >= 20000]['days_employed'].mean()

365004.3099162686

In [27]:
median_days_employed = df[df['days_employed'] <= 20000]['days_employed'].median()

In [28]:
median_days_employed

1629.9978622545889

In [29]:
def do_med(days):
    if days >= 20000:
        return median_days_employed
    else:
        return  days

In [30]:
df['days_employed'] = df['days_employed'].apply(do_med)

df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,F,сотрудник,0,253875.639453,недвижимость
1,1,4024.803754,36,1,0,F,сотрудник,0,112080.014102,автомобиль
2,0,5623.422610,33,1,0,M,сотрудник,0,145885.952297,недвижимость
3,3,4124.747207,32,1,0,M,сотрудник,0,267628.550329,образование
4,0,1629.997862,53,1,1,F,пенсионер,0,158616.077870,свадьба


In [31]:
min(df['dob_years'])

0

In [32]:
max(df['dob_years'])

75

In [33]:
df[df['dob_years'] < 18]['dob_years'].count()

101

In [34]:
df = df.loc[df['dob_years'] >= 18]

In [35]:
df[df['dob_years'] < 18]['dob_years'].count()

0

In [36]:
df['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

In [37]:
df['children'] = df['children'].replace(-1, 0)

In [38]:
df['children'].unique()

array([ 1,  0,  3,  2,  4, 20,  5], dtype=int64)

In [39]:
df[df['children'] == 20]['children'].count()

75

In [40]:
df = df.loc[df['children'] < 20]

In [41]:
df['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

In [42]:
df.isna().sum()

children               0
days_employed       2101
dob_years              0
education_id           0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2101
purpose                0
dtype: int64

In [43]:
df['days_employed'] = df['days_employed'].fillna(median_days_employed)

df.isna().sum()

children               0
days_employed          0
dob_years              0
education_id           0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2101
purpose                0
dtype: int64

In [44]:
median_total_income = df['total_income'].median()

df['total_income'] = df['total_income'].fillna(median_total_income)

df.isna().sum()

children            0
days_employed       0
dob_years           0
education_id        0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [45]:
df['total_income'] = df['total_income'].astype('int')

df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,недвижимость
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,автомобиль
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,недвижимость
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,образование
4,0,1629.997862,53,1,1,F,пенсионер,0,158616,свадьба


In [46]:
df['debt'].value_counts()

debt
0    19569
1     1725
Name: count, dtype: int64

In [47]:
def income_total(income):
    if income <= 30000:
        return 'E'
    elif income <= 50000:
        return 'D'
    elif income <= 200000:
        return 'C'
    elif income <= 1000000:
        return 'B'
    return 'A'

In [48]:
df['total_income_cat'] = df['total_income'].apply(income_total)
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_cat
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,недвижимость,B
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,автомобиль,C
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,недвижимость,C
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,образование,B
4,0,1629.997862,53,1,1,F,пенсионер,0,158616,свадьба,C


In [49]:
children_data_pivot = df.pivot_table(index = ['children'], columns = 'debt', values='purpose', aggfunc='count')
children_data_pivot

debt,0,1
children,,
0,13025.0,1059.0
1,4352.0,441.0
2,1845.0,194.0
3,301.0,27.0
4,37.0,4.0
5,9.0,NaN


In [53]:
children_data_pivot = children_data_pivot.fillna(0)
children_data_pivot

debt,0,1
children,,
0,13025.0,1059.0
1,4352.0,441.0
2,1845.0,194.0
3,301.0,27.0
4,37.0,4.0
5,9.0,0.0


In [54]:
children_data_pivot = children_data_pivot.reset_index()
children_data_pivot

debt,children,0,1
0,0,13025.0,1059.0
1,1,4352.0,441.0
2,2,1845.0,194.0
3,3,301.0,27.0
4,4,37.0,4.0
5,5,9.0,0.0


In [58]:
children_data_pivot['percent'] = children_data_pivot[1]/(children_data_pivot[0] + children_data_pivot[1]) * 100
children_data_pivot

debt,children,0,1,percent
0,0,13025,1059,7.519171
1,1,4352,441,9.200918
2,2,1845,194,9.514468
3,3,301,27,8.231707
4,4,37,4,9.756098
5,5,9,0,0.000000


In [59]:
children_data_pivot[0] = children_data_pivot[0].astype('int')
children_data_pivot[1] = children_data_pivot[1].astype('int')

children_data_pivot

debt,children,0,1,percent
0,0,13025,1059,7.519171
1,1,4352,441,9.200918
2,2,1845,194,9.514468
3,3,301,27,8.231707
4,4,37,4,9.756098
5,5,9,0,0.000000


In [ ]:
sns.barplot[x = 'children', y = 'parent'

In [61]:
family_data_pivot = df.pivot_table(index = ['family_status_id'], columns = 'debt', values='purpose', aggfunc='count')
family_data_pivot = family_data_pivot.reset_index()
family_data_pivot

debt,family_status_id,0,1
0,0,11323,924
1,1,3746,383
2,2,888,62
3,3,1099,84
4,4,2513,272


In [60]:
purpose_data_pivot = df.pivot_table(index = ['purpose'], columns = 'debt', values='purpose', aggfunc='count')

purpose_data_pivot

ValueError: Grouper for 'purpose' not 1-dimensional